In [201]:
import requests
import numpy as np
from math import log, exp
from scipy.optimize import linear_sum_assignment
import time

In [202]:
url = "https://api.twelvedata.com/price"

personal = "ef3e1be5126a41c0a9d98b02f4a7ce38"
uni = "b86172a213a240fca42701a5e89e0dbe"

def get_price(curr1, curr2, key):
    if curr1 == curr2:
        return 1
    else:
        try:
            resp = requests.request("GET", url=f"{url}?symbol={curr1}/{curr2}&apikey={key}").json()['price']
            return float(resp)
        except:
            resp = requests.request("GET", url=f"{url}?symbol={curr2}/{curr1}&apikey={key}").json()['price']
            return 1/float(resp)

In [203]:
class trader:
    def __init__(self, name, curr, home_curr, start_amount):
        self.name = name
        self.last_roi = 0
        self.curr = np.array(curr)
        self.volume = self.make_volume(home_curr, start_amount)
        self.fees = 0

    def make_volume(self, home_curr, start_amount):
        volume = dict()
        for c in self.curr:
            volume[c] = 0
        volume[home_curr] = start_amount
        return volume
    
    def get_M(self):

        self.P = np.matrix([[0.0 for _ in self.curr]for _ in self.curr], dtype=float)
        self.M = np.matrix([[0.0 for _ in self.curr]for _ in self.curr], dtype=float)
        C = np.matrix([[False for _ in self.curr]for _ in self.curr], dtype=bool)

        for r, curr1 in enumerate(self.curr):
            for c, curr2 in enumerate(self.curr):
                if C[r, c] == False:
                    p = get_price(curr1, curr2, self.key)
                    if p != 0:
                        self.P[r, c] = float(p) 
                        self.P[c, r] = float(1/p)
                        self.M[r, c] = log(float(p))
                        self.M[c, r] = log(float(1/p))
                    else:
                        self.P[r, c] = 1
                        self.P[c, r] = 1
                        self.M[r, c] = 0.0
                        self.M[c, r] = 0.0
                    C[r, c] = C[c, r] = True  

        return self.M

    def set_key(self, key):
        self.key = key

    def trade(self):
        self.get_M()
        row_ind, col_ind = linear_sum_assignment(self.M, maximize=True)
        
        self.last_roi = exp(self.M[row_ind, col_ind].sum())
        
        if self.last_roi > 1 + self.fees:
            child = {k:v for k,v in zip(row_ind, col_ind)}

            visited = set()
            self.orders(child, visited)

    
    def orders(self, child, visited, p = 0):
        if p not in visited:
            visited.add(p)

            # print(f"{self.curr[p]}: {self.volume[self.curr[p]]}", end = " -> ")

            self.volume[self.curr[child[p]]] += self.volume[self.curr[p]] * self.P[p, child[p]]
            self.volume[self.curr[p]] = 0

            # print(f"{self.curr[child[p]]}: {self.volume[self.curr[child[p]]]}")

            self.orders(child, visited, child[p])
        return



In [207]:

AUD = [1000]
hunter = trader("hunter", ["AUD", "BTC", "ETH", "USD"], 'AUD', AUD[0])
hunter.set_key(uni)

start = time.time()
print(time.strftime("%H:%M:%S"), "> initial investment (AUD):", AUD[0])

while True:
    hunter.trade()
    print(f"+{str(time.time() - start)[:5]} > +{hunter.volume['AUD'] - AUD[-1]}", "total:", hunter.volume['AUD'])
    AUD.append(hunter.volume['AUD'])
    time.sleep(60)

22:52:44 > initial investment (AUD): 1000
+3.404 > +5.836420819666614 total: 1005.8364208196666
+66.79 > +6.177814926417682 total: 1012.0142357460843
+130.1 > +7.002987567524087 total: 1019.0172233136084
+193.5 > +10.144851419756378 total: 1029.1620747333648
+256.8 > +10.516164137615306 total: 1039.67823887098
+320.4 > +4.114511692793258 total: 1043.7927505637733
+383.8 > +5.475386790575158 total: 1049.2681373543485
+447.1 > +7.554513911203458 total: 1056.822651265552
+510.6 > +13.367235976509846 total: 1070.1898872420618
+573.9 > +4.952394040093168 total: 1075.142281282155
+637.2 > +5.213525149934412 total: 1080.3558064320894
+700.6 > +-1080.3558064320894 total: 0


KeyboardInterrupt: 

In [ ]:
# big man...